# Alle Karteringen omzetten

Dit notebook voert alle karteringen via veg2hab naar een habitattypekartering. Instructies:

- Kopieer de provinciefolders uit de OneDrive: `Spheer AI/Spheer AI - General/Projecten/Veg_2_Hab/Data/Vegetatiekarteringen` naar `testing/vegetatiekarteringen` (of naar iets anders, dan moet je `originele_karteringen_prefix` aanpassen onder **config**)
- Eerst moeten sommige shapefiles opgeschoond worden. Dit kan met het `handmatig_opschonen_shapefiles.ipynb` notebook. Run deze als dat niet al eens gedaan is.
- Zorg dat de paden `originele_karteringen_prefix` en `opgeschoonde_shapefiles_prefix` kloppen en overeenkomen met dezelfde paden in het `handmatig_opschonen_shapefiles.ipynb`. Deze prefixes zullen voor de paden vermeld in de spreadsheets geplakt worden.
- Run de cellen. Deze zullen:
  - Imports importen en het bestaan van de paden controleren. 
  - De voor het omzetten benodigde klassen instantieren; de was-wordt lijst, de definitietabel en de FGR kaart.
  - Spreadsheets met daarin configuraties voor iedere kartering inlezen.
  - Alle karteringen inladen in een grote dict.
  - De volgende stappen een voor een op iedere kartering uitvoeren:
    - De was-wordt lijst toepassen (toevoegen van VvN codes aan karteringen met enkel SBB).
    - De definitietabel toepassen (het opstellen van habitatvoorstellen voor ieder vlak).
    - De mitsen checken (eventuele criteria in de habitatvoorstellen checken) en voor ieder vegetatietype een habitatkeuze maken.
    - De kartering formatten als een habitattypekartering en wegschrijven.  

### Imports etc

In [1]:
from pathlib import Path
import pandas as pd
import geopandas as gpd
import sys
sys.path.append('../')

from veg2hab.waswordtlijst import WasWordtLijst, opschonen_was_wordt_lijst
from veg2hab.definitietabel import DefinitieTabel, opschonen_definitietabel
from veg2hab.vegkartering import Kartering
from veg2hab.fgr import FGR

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 8)

originele_karteringen_prefix = '../testing/vegetatiekarteringen/'
opgeschoonde_shapefiles_prefix = '../testing/opgeschoonde_shapefiles/'
output_prefix = '../testing/omgezette_vegetatiekarteringen/'

assert Path(originele_karteringen_prefix).exists(), \
    f"De map {originele_karteringen_prefix} bestaat niet. Zet hier de vegetatiekarteringen uit de onedrive in."

assert Path(opgeschoonde_shapefiles_prefix).exists(), \
    f"De map {opgeschoonde_shapefiles_prefix} bestaat niet. Maak deze eerst aan met het handmatig_opschonen_shapefiles.ipynb notebook."


/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


### Omzetclasses instantieren

In [2]:
path_in_wwl = Path("../data/5. Was-wordt-lijst-vegetatietypen-en-habitattypen-09-02-2021.xlsx")
path_out_wwl = Path("../testing/opgeschoonde_waswordt.xlsx")
opschonen_was_wordt_lijst(path_in_wwl, path_out_wwl)
wwl = WasWordtLijst.from_excel(path_out_wwl)

path_in_dt = Path("../data/definitietabel habitattypen (versie 24 maart 2009)_0.xls")
path_in_mitsjson = Path("../data/mitsjson.csv")
path_out_dt = Path("../testing/opgeschoonde_definitietabel.xlsx")
opschonen_definitietabel(path_in_dt, path_in_mitsjson, path_out_dt)
dt = DefinitieTabel.from_excel(path_out_dt)

fgr = FGR(Path("../data/bronbestanden/FGR.json"))

/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Alle SBB codes zijn valide
Alle VvN codes zijn valide
Alle SBB codes zijn valide
Alle VvN codes zijn valide
Alle SBB codes zijn valide
Alle VvN codes zijn valide


### Inladen benodigde informatie

**Overzicht_vegetatiekarteringen.xlsx** is uit de drive en weet van karteringen de provincie, naam, shapefile pad en of het een access of een shapefile kartering is.

**accesskarteringinfo.xlsx** bevat voor de access karteringen de informatie die nodig is om een access kartering in te laden.

**shapefilekarteringinfo.xlsx** bevat voor de shapefile karteringen de informatie die nodig is om een shapefile kartering in te laden.


In [3]:
# info over de karteringen
overzicht = pd.read_excel(
    "../data/Overzicht_vegetatiekarteringen.xlsx", 
    sheet_name="Vegetatiekarteringen", 
    engine='openpyxl',
    usecols=[
        "provincie", 
        "naam_kartering", 
        "path_shapes", 
        "landelijke_typologie beschikbaar?", 
        "locatie_landelijke_typologie",
    ],
    dtype="string"
)

# benodigde info voor de access karteringen
access_info = pd.read_excel(
    "../data/accesskarteringinfo.xlsx",
    engine='openpyxl',
    usecols=[
        "naam_kartering",
        "path_csvs",
        "elm_id_name",
    ],
    dtype="string"
)

# benodigde info voor de shapefile karteringen
shape_info = pd.read_excel(
    "../data/shapefilekarteringinfo.xlsx",
    engine='openpyxl',
    usecols=[
        "naam_kartering",
        "ElmID_col",
        "datum_col",
        "opmerking_col",
        "vegtype_col_format",
        "sbb_of_vvn",
        "SBB_col",
        "VvN_col",
        "split_char",
        "perc_col",
    ],
    dtype="string"
)
# Omzetten van pd.NA naar None om te voorkomen dat we overal "if not pd.isna()" moeten doen
# NOTE: dit naar vegkaartering.py?
shape_info = shape_info.applymap(lambda x: None if pd.isna(x) else x)

overzicht = overzicht[overzicht["landelijke_typologie beschikbaar?"] == "Ja"]
# omzetten van ./PROVINCIE/gebied/gebied.shp naar ../testing/opgeschoonde_shapefiles/PROVINCIE/gebied/gebied.shp
overzicht.path_shapes = overzicht.path_shapes.apply(lambda x: opgeschoonde_shapefiles_prefix + x[2:])

access_overzicht = overzicht[overzicht["locatie_landelijke_typologie"] == "access"]
access_overzicht = access_overzicht.join(access_info.set_index("naam_kartering"), on="naam_kartering")
# omzetten van ./PROVINPCIE/gebied/gebied.shp naar ../testing/vegetatiekarteringen/PROVINCIE/gebied/gebied.shp
access_overzicht.path_csvs = access_overzicht.path_csvs.apply(lambda x: originele_karteringen_prefix + x[2:])

shapefile_overzicht = overzicht[overzicht["locatie_landelijke_typologie"] == "in shapes"]
shapefile_overzicht = shapefile_overzicht.join(shape_info.set_index("naam_kartering"), on="naam_kartering")

### Karteringen inladen

Hier worden, adhv configuraties uit de spreadsheets, alle karteringen ingeladen. Als er een probleem is met een kartering, wordt deze overgeslagen en wordt een melding gegeven.

In [8]:
karteringen = {}
mislukte_karteringen = []

# for row in access_overzicht.itertuples():
#     if row.naam_kartering != 'NM_Fochteloerveen2013_14':
#         continue
#     print(f"------ Inlezen {row.naam_kartering}")
#     try:
#         karteringen[row.naam_kartering] = Kartering.from_access_db(Path(row.path_shapes), row.elm_id_name, Path(row.path_csvs))
        
#     except Exception as e:
#         print(e)
#         print(f"!!! {row.naam_kartering} wordt overgeslagen !!!")
#         mislukte_karteringen.append(row.naam_kartering)
#         continue

#     print(f"Gelukt!")

for row in shapefile_overzicht.itertuples():
    print(f"------ Inlezen {row.naam_kartering}")
    try:
        karteringen[row.naam_kartering] = Kartering.from_shapefile(
            Path(row.path_shapes),
            row.ElmID_col,
            vegtype_col_format=row.vegtype_col_format,
            sbb_of_vvn=row.sbb_of_vvn,
            VvN_col=row.VvN_col,
            SBB_col=row.SBB_col, 
            split_char=row.split_char, 
            datum_col=row.datum_col, 
            opmerking_col=row.opmerking_col,
            perc_col=row.perc_col  
        )
    
    except Exception as e:
        print(e)
        print(f"!!! {row.naam_kartering} wordt overgeslagen !!!")
        mislukte_karteringen.append(row.naam_kartering)
        continue

    print(f"Gelukt!")

if len(mislukte_karteringen) > 0:
    print(f"!!! De volgende karteringen zijn niet ingelezen !!!")
    print(mislukte_karteringen)

------ Inlezen BG_Holtingerveld_bossen2016


ValueError: cannot reindex from a duplicate axis

### Karteringen omzetten

In [ ]:
for key, kartering in karteringen.items():
    print(f"------ Waswordtlijst toepassen {key}")
    try:
        kartering.apply_wwl(wwl)    
    except Exception as e:
        print(e)
        print(f"!!! Fout bij {key} !!!")
        continue
    print(f"Gelukt!")

------ Waswordtlijst toepassen NM_Fochteloerveen2013_14
Gelukt!


/home/jordydelange/git/veg2hab/veg2hab/vegkartering.py:860: UserWarning: Er zijn al VvN aanwezig in de kartering. De was-wordt lijst wordt niet toegepast.
  "Er zijn al VvN aanwezig in de kartering. De was-wordt lijst wordt niet toegepast."


In [ ]:
for key, kartering in karteringen.items():
    print(f"------ Definitietabel toepassen {key}")
    try:  
        kartering.apply_deftabel(dt)        
    except Exception as e:
        print(e)
        print(f"!!! Fout bij {key} !!!")
        continue
    print(f"Gelukt!")

------ Definitietabel toepassen NM_Fochteloerveen2013_14
Gelukt!


In [ ]:
for key, kartering in karteringen.items():
    print(f"------ Criteria checken {key}")
    try:
        kartering.check_mitsen(fgr)    
    except Exception as e:
        print(e)
        print(f"!!! Fout bij {key} !!!")
        continue
    print(f"Gelukt!")

------ Criteria checken NM_Fochteloerveen2013_14
Gelukt!


### Karteringen wegschrijven

In [ ]:
for gebied in (karteringen.keys()):
    print(gebied)
    # Vervangen van de shapefile path naar de output path
    p = Path(overzicht[overzicht.naam_kartering == gebied].path_shapes.str.replace(opgeschoonde_shapefiles_prefix, output_prefix).iloc[0])
    p.parent.mkdir(parents=True, exist_ok=True)
    karteringen[gebied].final_format_to_file(p)

NM_Fochteloerveen2013_14
